In [31]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.weightstats as ws
from scipy.stats import norm, ttest_ind

In [32]:
df= pd.read_csv("Los_Angeles_Long_Beach_Anaheim_2.csv")

In [33]:
# Task 3.1 
# Change aplha 1.5 => 1

def find_outliers(df, column_name, alpha=1):
     column_df = df[column_name]
     Q1 = column_df.quantile(0.25)
     Q3 = column_df.quantile(0.75)
     IQR  = Q3 -Q1 

     low_range = Q1 - IQR * alpha
     upper_range = Q3 +  IQR * alpha

     outliers = pd.DataFrame(columns=df.columns)

     for _, r in df.iterrows():
        if r[column_name] < low_range or r[column_name] > upper_range:
            outliers = pd.concat([outliers, r.to_frame().T])

     return  outliers


                             


In [34]:
column_list= ['CPI_all_items', 'Rent_of_Primary_residence',
       'Monthly_Housing_Cost', 'CPI_Energy', 'US_Dollar_Purchasing_power']
for column in column_list:
    outliers = find_outliers(df, column)
    print(f"The number of outliers of column {column}:  {len(outliers)}")
    print(f"The outliers of column {column}:")
    select_df = outliers[['DATE', column]] 
    
    if len(outliers) != 0:
        print(select_df)
    print()

    # delete outliers 
    
    df.loc[outliers.index, column] = np.nan


# df.to_csv("data_with_outlier.csv", index=False)

The number of outliers of column CPI_all_items:  4
The outliers of column CPI_all_items:
           DATE CPI_all_items
308  2022-09-01       296.539
309  2022-10-01       297.987
310  2022-11-01       298.598
311  2022-12-01        298.99

The number of outliers of column Rent_of_Primary_residence:  1
The outliers of column Rent_of_Primary_residence:
           DATE Rent_of_Primary_residence
286  2020-11-01                      40.0

The number of outliers of column Monthly_Housing_Cost:  0
The outliers of column Monthly_Housing_Cost:

The number of outliers of column CPI_Energy:  0
The outliers of column CPI_Energy:

The number of outliers of column US_Dollar_Purchasing_power:  0
The outliers of column US_Dollar_Purchasing_power:



In [7]:
# # Task 3.2 


# def linear_interpolation(df):
#     for column in df.columns:
#         column_has_nan = df[column].isnull().any()
#         if column_has_nan:
#             for i, value in enumerate(df[column]):
#                 if np.isnan(value):
#                     # Find previous value and index
#                     prev_val = None
#                     prev_idx = i - 1
#                     while prev_idx >= 0:
#                         if not np.isnan(df.at[prev_idx, column]):
#                             prev_val = df.at[prev_idx, column]
#                             break
#                         prev_idx -= 1

#                     # Find next value and index
#                     next_val = None
#                     next_idx = i + 1
#                     while next_idx < len(df[column]):
#                         if not np.isnan(df.at[next_idx, column]):
#                             next_val = df.at[next_idx, column]
#                             break
#                         next_idx += 1

#                     if prev_val is not None and next_val is not None:
#                         # Calculate slope and missing value
#                         slope = (next_val - prev_val) / (next_idx - prev_idx)
#                         missing_value = prev_val + slope * (i - prev_idx)
#                         df.at[i, column] = missing_value
#                     elif prev_val is not None:
#                         df.at[i, column] = prev_val
#                     elif next_val is not None:
#                         df.at[i, column] = next_val
#     return df

# df = linear_interpolation(df)
# df.to_csv("completed_dataset.csv", index=False)



In [35]:
# Task 3.2 

# We replace the last value of the CPI_all_items  because the vaule is null 
df.loc[311, 'CPI_all_items'] =  298.900
# update_values = ['2022-12-01', 298.900, 385.649, 310.725, 287.176, 33.7]



def show_missing_values(df):
    missing_values = df.isna()
    for column in missing_values.columns:
        print(f"Column '{column}':")
        missing_rows = missing_values[column][missing_values[column]]
        for index in missing_rows.index:
            if not np.isnan(df.loc[index, column]):
                continue
            left_index  = index -1
            left_val= df.loc[left_index, column]

            right_index =index+1
            right_val =df.loc[right_index, column]
            roop = True
            while roop:
                if not np.isnan(right_val): 
                    roop = False
                else : 
                    right_index =right_index+1
                    right_val =df.loc[right_index, column]
            slope = (right_val - left_val) / (right_index - left_index)
            # missing_value = left_val + slope * (index - left_index)
            # df.loc[index, column] = missing_value
            # print(f"    - Replace the Missing value to {missing_value}")

            while index <  right_index:
                missing_value = left_val + slope * (index - left_index)
                df.loc[index, column] = missing_value
                print(f"    - Find Missing value at index {index} and Replace the Missing value to {missing_value}")
                index+= 1
                
            
# 결측치 위치 확인 및 출력
show_missing_values(df)

Column 'DATE':
Column 'CPI_all_items':
    - Find Missing value at index 308 and Replace the Missing value to 296.215
    - Find Missing value at index 309 and Replace the Missing value to 297.11
    - Find Missing value at index 310 and Replace the Missing value to 298.005
Column 'Rent_of_Primary_residence':
    - Find Missing value at index 257 and Replace the Missing value to 358.4205
    - Find Missing value at index 269 and Replace the Missing value to 370.46900000000005
    - Find Missing value at index 273 and Replace the Missing value to 374.8265
    - Find Missing value at index 286 and Replace the Missing value to 384.03499999999997
Column 'Monthly_Housing_Cost':
    - Find Missing value at index 258 and Replace the Missing value to 299.297
    - Find Missing value at index 262 and Replace the Missing value to 300.81399999999996
Column 'CPI_Energy':
    - Find Missing value at index 268 and Replace the Missing value to 257.8195
    - Find Missing value at index 273 and Replac

In [36]:
#step 3.3
def compare_means_one_sided(df, column_name, year1, year2,alpha = 0.05 ):
    # Hypothesis:
    # Null Hypothesis (H0): Mean of column in year1 <= Mean of column in year2
    # Alternative Hypothesis (H1): Mean of column in year1 > Mean of column in year2 (if alternative='greater')
    # Ensure 'date' column is in datetime format
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['year'] = df['DATE'].dt.year
    df['month'] = df['DATE'].dt.month

    # Filter data for each year
    df_year1 = df[df['year'] == year1]
    df_year2 = df[df['year'] == year2]
    
    
    # Calculate monthly mean for each year
    monthly_mean_year1 = df_year1.groupby('month')[column_name].mean()
    monthly_mean_year2 = df_year2.groupby('month')[column_name].mean()
     #  we know MLE of Poisson distributed = sample mean of data  

    lambda_hat_x , lambda_hat_y  = monthly_mean_year1.mean(), monthly_mean_year2.mean()

    std1, std2 = monthly_mean_year1.std(), monthly_mean_year2.std()
    n1, n2 = len(monthly_mean_year1), len(monthly_mean_year2)
   
    critical_value = round(stats.norm.ppf(1 - alpha),2)

    # Wald's Test
    
    #  we know variance of MLE of Poisson =  lambda ,so  

    wald_stat = (lambda_hat_x - lambda_hat_y) / np.sqrt(lambda_hat_x + lambda_hat_y) 
    # wald_stat = (mean1 - mean2) / np.sqrt(std1**2/n1 + std2**2/n2)
    # wald_p_value = norm.sf(wald_stat) 


    # Z-test
    z_stat = (lambda_hat_x - lambda_hat_y) / np.sqrt(std1**2/n1 + std2**2/n2)

    # t-test
    t_stat, _ = ttest_ind(monthly_mean_year1, monthly_mean_year2, equal_var=False)

    print(f"Results for {column_name} between {year1} and {year2}:")
    print(f"Null Hypothesis (H0): Mean of {column_name} in {year1} <= Mean of {column_name} in {year2}")
    print(f"Alternative Hypothesis (H1): Mean of {column_name} in {year1} > Mean of {column_name} in {year2}") 
    print(f"\nWald's Test: Statistic = {wald_stat}")
    print(f"Z-test: Statistic = {z_stat}")
    print(f"t-test: Statistic = {t_stat}")
    print()

    for test_name, stat in [("Wald's Test", abs(wald_stat)), ("Z-test", abs(z_stat)), ("t-test", abs(t_stat))]:
        if stat > critical_value:
            print(f"{test_name}: Null hypothesis is rejected ( stat of {test_name}: {stat} > critical_value : {critical_value}). There is a significant difference between the means.")
        else:
            print(f"{test_name}: Null hypothesis is not rejected (stat of {test_name} {stat} <= critical_value : {critical_value}). There is no significant difference between the means.")
        print()

In [37]:
def compare_means_two_sided(df, column_name, year1, year2,alpha = 0.05 ):
    # Hypothesis:
    # Null Hypothesis (H0): Mean of column in year1 = Mean of column in year2
    # Alternative Hypothesis (H1): Mean of column in year1 != Mean of column in year2 
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['year'] = df['DATE'].dt.year
    df['month'] = df['DATE'].dt.month

    # Filter data for each year
    df_year1 = df[df['year'] == year1]
    df_year2 = df[df['year'] == year2]
    
    
    # Calculate monthly mean for each year
    monthly_mean_year1 = df_year1.groupby('month')[column_name].mean()
    monthly_mean_year2 = df_year2.groupby('month')[column_name].mean()
     #  we know MLE of Poisson distributed = sample mean of data  

    lambda_hat_x , lambda_hat_y  = monthly_mean_year1.mean(), monthly_mean_year2.mean()

    std1, std2 = monthly_mean_year1.std(), monthly_mean_year2.std()
    n1, n2 = len(monthly_mean_year1), len(monthly_mean_year2)
   
    critical_value = round(stats.norm.ppf(1 - alpha/2),2)

    # Wald's Test
    
    #  we know variance of MLE of Poisson =  lambda ,so  

    wald_stat = (lambda_hat_x - lambda_hat_y) / np.sqrt(lambda_hat_x + lambda_hat_y) 


    # Z-test
    z_stat = (lambda_hat_x - lambda_hat_y) / np.sqrt(std1**2/n1 + std2**2/n2)

    # t-test
    t_stat, _ = ttest_ind(monthly_mean_year1, monthly_mean_year2, equal_var=False)

    print(f"Results for {column_name} between {year1} and {year2}:")
    print(f"Null Hypothesis (H0): Mean of {column_name} in {year1} = Mean of {column_name} in {year2}")
    print(f"Alternative Hypothesis (H1): Mean of {column_name} in {year1} != Mean of {column_name} in {year2}") 
    print(f"\nWald's Test: Statistic = {wald_stat}")
    print(f"Z-test: Statistic = {z_stat}")
    print(f"t-test: Statistic = {t_stat}")
    print()

    for test_name, stat in [("Wald's Test", abs(wald_stat)), ("Z-test", abs(z_stat)), ("t-test", abs(t_stat))]:
        if stat > critical_value:
            print(f"{test_name}: Null hypothesis is rejected ( stat of {test_name}: {stat} > critical_value : {critical_value}). There is a significant difference between the means.")
        else:
            print(f"{test_name}: Null hypothesis is not rejected (stat of {test_name} {stat} <= critical_value : {critical_value}). There is no significant difference between the means.")
        print()

In [38]:


df['DATE'] = pd.to_datetime(df['DATE'])

compare_means_one_sided(df, "Rent_of_Primary_residence", 2020, 2021)
compare_means_two_sided(df, "Rent_of_Primary_residence", 2020, 2021)
compare_means_one_sided(df, "CPI_Energy", 2020, 2021)
compare_means_two_sided(df, "CPI_Energy", 2020, 2021)


Results for Rent_of_Primary_residence between 2020 and 2021:
Null Hypothesis (H0): Mean of Rent_of_Primary_residence in 2020 <= Mean of Rent_of_Primary_residence in 2021
Alternative Hypothesis (H1): Mean of Rent_of_Primary_residence in 2020 > Mean of Rent_of_Primary_residence in 2021

Wald's Test: Statistic = -0.2762005472840209
Z-test: Statistic = -6.139495456059521
t-test: Statistic = -6.139495456059521

Wald's Test: Null hypothesis is not rejected (stat of Wald's Test 0.2762005472840209 <= critical_value : 1.64). There is no significant difference between the means.

Z-test: Null hypothesis is rejected ( stat of Z-test: 6.139495456059521 > critical_value : 1.64). There is a significant difference between the means.

t-test: Null hypothesis is rejected ( stat of t-test: 6.139495456059521 > critical_value : 1.64). There is a significant difference between the means.

Results for Rent_of_Primary_residence between 2020 and 2021:
Null Hypothesis (H0): Mean of Rent_of_Primary_residence in

In [39]:
#step 3.4
import numpy as np

# Normalize the data by converting the columns to percentage change
# def percent_change(column):
#     result = [None]
#     for i in range(1, len(column)):
#         result.append((column[i] - column[i-1])/column[i-1])
#     return result

# df['CPI_all_items_per_change'] = percent_change(df['CPI_all_items'])
# df['Rent_of_Primary_residence_per_change'] = percent_change(df['Rent_of_Primary_residence'])
# df['Monthly_Housing_Cost_per_change'] = percent_change(df['Monthly_Housing_Cost'])
# df['CPI_Energy_per_change'] = percent_change(df['CPI_Energy'])
# df['US_Dollar_Purchasing_power_per_change'] = percent_change(df['US_Dollar_Purchasing_power'])

# Filter the data between 2018-2020
filtered_df = df[(df['year'] >= 2018) & (df['year'] <= 2020)]

# housing_data = filtered_df['CPI_all_items_per_change'].values
# rent_data = filtered_df['Rent_of_Primary_residence_per_change'].values

housing_data = filtered_df['CPI_all_items'].values
rent_data = filtered_df['Rent_of_Primary_residence'].values


In [40]:
from scipy.stats import poisson, geom, binom, ksone, kstwo # to get cdf of distributions
from scipy.stats import ks_1samp, ks_2samp
def ks_test(sample_data, cdf_function):
    n = len(sample_data)
    sorted_sample_data = np.sort(sample_data)
    cdf_sample_data = np.arange(0, n + 1) / n
    cdf_theo = np.array([cdf_function(x) for x in sorted_sample_data])
    ks_stat = 0
    for i in range(len(cdf_theo)):
        d_neg = np.abs(cdf_theo[i]-cdf_sample_data[i])
        d_pos = np.abs(cdf_theo[i]-cdf_sample_data[i+1])
        if d_neg > ks_stat:
            ks_stat = d_neg
        if d_pos > ks_stat:
            ks_stat = d_pos
    p_value = ksone.sf(ks_stat, len(sample_data))
    return ks_stat, p_value

def ks_test_with_distributions(MME_param_sample, sample_data, critical_value=0.05):
    test_distributions = []
    # Binomial distribution 
    # TODO MME formula needs to be reviewed
    mean = np.mean(MME_param_sample) 
    squared_mean = np.mean(MME_param_sample ** 2)
    binomial_p_mme = 1 - (squared_mean / mean)
    binomial_n_mme = mean / binomial_p_mme
    print(binomial_n_mme, binomial_p_mme)
    binomial_dist = binom(binomial_n_mme, binomial_p_mme)
    if binomial_n_mme < 0 or binomial_p_mme < 0:
        print('KS test with binomial distribution cannot be conducted. MME for binomial distribution yielded negative values')
    else:
        ks_stat_binomial, p_value_binomial = ks_test(sample_data, binomial_dist.cdf)
        test_distributions.append(("Binomial", ks_stat_binomial, p_value_binomial))

    # Poisson distribution
    poisson_mme = mean
    print(poisson_mme)
    poisson_dist = poisson(poisson_mme)
    ks_stat_poisson, p_value_poisson = ks_test(sample_data, poisson_dist.cdf)
    test_distributions.append(("Poisson", ks_stat_poisson, p_value_poisson))

    # Geometric distribution
    geometric_mme = 1 / mean
    print(geometric_mme)
    geometric_dist = geom(geometric_mme)
    ks_stat_geometric, p_value_geometric = ks_test(sample_data, geometric_dist.cdf)
    test_distributions.append(("Geometric", ks_stat_geometric, p_value_geometric))

    # Print results
    print(f"Kolmogorov-Smirnov Test Results:\n")
    for dist_name, ks_stat, p_value in test_distributions:
        print(f"{dist_name}: KS Statistic = {ks_stat}")
        print(f"{dist_name}: P-value = {p_value}")
        if p_value > critical_value:
            print(f"Null hypothesis is rejected (p_value {p_value} > {critical_value}). Two samples are not likely come from {dist_name} distribution.")
        else:
            print(f"Null hypothesis is not rejected (p_value {p_value} <= {critical_value}). Two samples are likely come from {dist_name} distribution.")
        print()
ks_test_with_distributions(MME_param_sample = housing_data, sample_data = rent_data)   

# # Estimate parameters using MME
# mu = np.mean(housing_data)  # for Poisson
# gp = 1 / np.mean(housing_data)  # for Geometric
# mean = np.mean(housing_data) 
# squared_mean = np.mean(housing_data ** 2)
# p = 1 - (squared_mean / mean)
# n = mean / p

# # Perform 1-sample KS tests
# for dist in [poisson(mu), geom(gp), binom(n, p)]:
#     statistic, pvalue = ks_1samp(x = rent_data, cdf = dist.cdf)
#     print(statistic, pvalue)

-1.0037377162156136 -254.2502679838659
KS test with binomial distribution cannot be conducted. MME for binomial distribution yielded negative values
255.20058333333333
0.0039184863409729665
Kolmogorov-Smirnov Test Results:

Poisson: KS Statistic = 0.9999999979075557
Poisson: P-value = 3.495753772e-313
Null hypothesis is not rejected (p_value 3.495753772e-313 <= 0.05). Two samples are likely come from Poisson distribution.

Geometric: KS Statistic = 0.7508920738365887
Geometric: P-value = 7.979114937813692e-22
Null hypothesis is not rejected (p_value 7.979114937813692e-22 <= 0.05). Two samples are likely come from Geometric distribution.



In [41]:
import numpy as np
# Null Hypothesis -> Two datasets are from the same distribution.

def two_sample_ks_test(d1, d2):
    d1 = sorted(d1)
    d2 = sorted(d2)
    merged_d = []
    d1_pointer = 0
    d2_pointer = 0
    while d1_pointer < len(d1) or d2_pointer < len(d2):
        if d1_pointer == len(d1):
            merged_d.append(d2[d2_pointer])
            d2_pointer += 1
        elif d2_pointer == len(d2):
            merged_d.append(d1[d1_pointer])
            d1_pointer+=1
        else:
            d1_val = d1[d1_pointer]
            d2_val = d2[d2_pointer]
            if d1_val < d2_val:
                merged_d.append(d1_val)
                d1_pointer+=1
            elif d1_val > d2_val:
                merged_d.append(d2_val)
                d2_pointer+=1
            else:
                merged_d.append(d1_val)
                d1_pointer+=1
                d2_pointer+=1
    F_d1 = []
    F_d2 = []
    d1_pointer = 0
    d2_pointer = 0
    ks_stat = 0
    for val in merged_d:
        if d1_pointer < len(d1) and d1[d1_pointer] == val:
            F_d1.append(F_d1[-1] + (1/len(d1)) if d1_pointer > 0 else (1/len(d1)))
            d1_pointer += 1
        else:
            F_d1.append(F_d1[-1] if d1_pointer > 0 else 0)

        if d2_pointer < len(d2) and d2[d2_pointer] == val:
            F_d2.append(F_d2[-1] + (1/len(d2)) if d2_pointer > 0 else (1/len(d2)))
            d2_pointer += 1
        else:
            F_d2.append(F_d2[-1] if d2_pointer > 0 else 0)
    for i in range(len(F_d1)):
        val = abs(F_d1[i]-F_d2[i])
        if val > ks_stat:
            ks_stat = val
    p = kstwo.sf(ks_stat, (len(d1)+len(d2))/2)
    return ks_stat, p

# Perform 2-sample KS test
ks_stat, p_value = two_sample_ks_test(housing_data, rent_data)
print(f"2-sample KS Test: KS Statistic = {ks_stat}")
print(f"2-sample KS Test: P-value = {p_value}")
if p_value > 0.05:
    print("Null hypothesis is rejected (ks_stat > 0.05). Two samples are likely come from different distributions.")
else:
    print("Null hypothesis is not rejected (ks_stat <= 0.05). Two samples are likely come from the same distribution.")
# print(ks_2samp(housing_data,rent_data, method='exact'))


2-sample KS Test: KS Statistic = 1.0000000000000002
2-sample KS Test: P-value = 0.0
Null hypothesis is not rejected (ks_stat <= 0.05). Two samples are likely come from the same distribution.


In [42]:
# Permutation test
# using code from a5.q5 Dukyoung Eom  
# Null hypothesis -> Two samples are from the same distribution
import random

def permutation_test(dist1, dist2, num_iters=1000):

    dist1 = list(dist1)
    dist2 = list(dist2)
    T_obs = abs(np.average(dist1)-np.average(dist2))
    Ti_count = 0
    permutation_results = set()
    while len(permutation_results) < num_iters:
        random_perm = tuple(random.sample(dist1+dist2, len(dist1) + len(dist2)))
        permutation_results.add(random_perm)
    permutation_results = list(permutation_results)
    for i in range(num_iters):
        sample_dist1 = permutation_results[i][:len(dist1)]
        sample_dist2 = permutation_results[i][len(dist1):]
        T_i = abs(np.average(sample_dist1)-np.average(sample_dist2))
        if T_i > T_obs:
            Ti_count += 1
    return Ti_count / num_iters

p_value = permutation_test(housing_data, rent_data)
print(f'Permutation test: p-value = {p_value:.4f}')

if p_value < 0.05:
    print("Null hypothesis is rejected (p_value < 0.05). Two samples are likely come from different distributions.")
else:
    print("Null hypothesis is not rejected (p_value >= 0.05). Two samples are likely come from the same distribution.")

Permutation test: p-value = 0.0000
Null hypothesis is rejected (p_value < 0.05). Two samples are likely come from different distributions.
